# **Trong công đoạn thu thập dữ liệu bao gồm 3 công đoạn chính:**
- Spotify API: Sử dụng thư viện Spotipy được hỗ trợ cho việc sử dụng API của Spotify một cách hiệu quả.
- Genius API: Được dùng để thu thập lyric và tag. Tuy nhiên, Genius API không cung cấp lyric mà phải thông qua một API khác. API này sử dụng genius id của một bài nhạc. Vì thế, ta sẽ dùng Genius API để lấy được genius id và released_date (ngày ra mắt của bài nhạc).
- Rapid API: API này dùng để lấy dữ liệu lyric cũng như tag (thể loại chính của bài nhạc) và tier (mức xếp hạng bài nhạc) từ genius id.

## 1) Sử dụng thư viện Spotipy:
**Ý tưởng**: Ý tưởng ban đầu của nhóm là chọn theo nhóm album hoặc theo tuyển tập nhạc. Tuy nhiên, việc này khá khó khi không thể biết được tên cụ thể của từng tuyển tập nhạc. Vì thế, nhóm chuyển sang ý tưởng random nhạc trên số lượng lớn. May mắn thay, dữ liệu thu được khá ổn.
**Các trục trặc**: Lúc đầu nhóm thu các features và lyric. Sau đó, không thể lấy lyric khi Spotify không còn hỗ trợ việc này. Vì thế nhóm xem xét tới Genius API, và cần thêm tên bài và tên nghệ sĩ.


In [ ]:
import pandas as pd

In [ ]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
import spotipy
import sys
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import random
import numpy as np

CLIENT_ID = '01df5da9702a44dd9e8ccfc0746055e7'
CLIENT_SECRET = 'de0273d1004843b683fd8053799bbb5a'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET))

### Hàm random alphabet để tìm kiếm:

In [ ]:
def get_random_search():
    characters = 'abcdefghijklmnopqrstuvwxyz'
    random_character = random.choice(characters)
    random_search = ''
    random_search = '%' + random_character + '%'
    return random_search

### Tìm 5000 track random:

In [ ]:
tracks = []
for i in range(0,1000):
  tracks.append(sp.search(get_random_search(),offset= random.randint(0, 500),limit=50))

### Lấy id của các track vào list track_ids

In [ ]:
track_df = pd.DataFrame()
for i in tracks:
    track_df = pd.concat([track_df,pd.json_normalize(i['tracks']['items'])],axis=0)

In [ ]:
track_ids = list(track_df.id.unique())

### Lấy các thuộc tính nhạc từ id của từng track (50 cái mỗi lần do spotify API limit = 50)

In [ ]:
features_df = []
for i in range(0,len(track_ids),50):
  features_df.append(sp.audio_features(track_ids[i:i+50]))

In [ ]:
flattened_list = [item for sublist in features_df for item in sublist]
filtered_data_list = [data for data in flattened_list if data is not None and data]
features_df = pd.DataFrame(filtered_data_list)

In [ ]:
features_df.to_csv('features_df.csv', index = False, encoding = 'utf-8')

In [ ]:
features_df = pd.read_csv("features_df.csv", encoding = 'utf-8')

### Cào thêm các cột popularity, song_name, artist_name cho data thêm phong phú để phân tích:

In [ ]:
popularity = []
song_name = []
artist_name = []
for id in features_df['id']:
  track = sp.track(id)
  artist_name.append(track['artists'][0]['name'])
  song_name.append(track['name'])
  popularity.append(track['popularity'])

In [ ]:
features_df['song_name'] = song_name
features_df['popularity'] = popularity
features_df['artist_name'] = artist_name

In [ ]:
features_df.to_csv('spotify.csv', index = False, encoding = 'utf-8')

## 2) Sử dụng Genius để tìm id và release_date từng track:
**Ý tưởng**: Sau khi lấy được dữ liệu sử dụng Spotipy, nhóm sử dụng Genius API. API này không như Spotify bị giới hạn số lần, nên việc lấy dữ liệu khá dễ dàng. Lúc đầu nhóm sử dụng để lấy id, tuy nhiên thông tin released_date cũng khá hữu dụng nên nhóm quyết định lấy thêm dữ liệu này.

**Các trục trặc**:Tuy nhiên, API này lại không hỗ trợ cho việc lấy lyric mà chỉ trả về thông tin liên quan tới bài nhạc.

In [ ]:
import requests
import json

def request_song_info(song_title, artist_name):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + '5l4zx-dfAx4KV5iNuGUQs2QJ_YsaVqIY_eStOF_CJLIISlt8jZcZkPaQmGCyES2_'}
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, params=data, headers=headers)

    return response

In [ ]:
id = []
date = []
count = 0
for song, artist in zip(features_df['song_name'], features_df['artist_name']):
  response = request_song_info(song, artist)
  if response.status_code == 200 :
    json = response.json()
    if(json['response']['hits'] != []):
      id.append(json['response']['hits'][0]['result']['id'])
      date.append(json['response']['hits'][0]['result']['release_date_components'])
    else:
      id.append('-1')
      date.append({'year': 0, 'month': 0, 'day': 0})
  else:
    id.append('-1')
    date.append({'year': 0, 'month': 0, 'day': 0})
  print(count + 1)
  count+=1

In [ ]:
features_df['genius_id'] = id
features_df['release_date'] = date

In [ ]:
features_df.to_csv('spotify_genius.csv',index = False,encoding = "utf-8")

## 3) Cào tags, lyrics, tier sử dụng API có trên Rapid API:
**Ý tưởng**: API trên trang cho phép khi đưa vào genius id thì sẽ trả về thông tin lời nhạc của bài cũng như các thể loại. Đây là API cần nhất để lấy được 2 dữ liệu mà nhóm cần, đó là lyric và tag (thể loại chính của nhạc).

**Trục trặc**: Vấn đề lớn nhất là API chỉ cho phép 100 request mỗi lần. Tức là chỉ 100 bài nhạc. Dữ liệu của nhóm thu được tới 5000 bài hơn. Chính vì thế, nhómm quyết định lấy những bài nhạc nổi (tức popularity >80) để giảm thiểu số lượng bài cũng như tập trung vào dữ liệu chất lượng.

Đọc data từ Genius API và lọc ra các bài popularity trên 80.

In [ ]:
features_df = pd.read_csv('spotify_genius.csv', encoding = 'utf-8')


In [ ]:
df = features_df

In [ ]:
c = df[180:]
c = c[c['popularity'] >= 80]
c.to_csv('popular_gt_80.csv',index = False)

### Từ 0 đến 399

0 den 100

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "8b7b8677cbmshda7d2be90521933p12dbbejsnf58fb3942d5b",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[:100]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  429
O


In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_100.csv',index = False)

100 toi 200

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "fa8d2e20bemsh0425391cf102695p16716bjsn007e194a764e",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[100:200]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  429
O


In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_200.csv',index = False)

200 toi 300

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "eb14d2c446mshe73641800cba6a7p159db1jsn5c0df1180378",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[200:300]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  429
O


In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_300.csv',index = False)

300 toi 400

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "d942db774bmshd921792eea11f3ep178f38jsn01d246d8f41f",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[300:400]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  429
O


In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_400.csv',index = False)

In [ ]:
df_500 = pd.read_csv('df_100.csv', encoding = 'utf-8')
df_600 = pd.read_csv('df_200.csv', encoding = 'utf-8')
df_700 = pd.read_csv('df_300.csv', encoding = 'utf-8')
df_800 = pd.read_csv('df_400.csv', encoding = 'utf-8')
full = pd.concat([df_100,df_200,df_300,df_400],axis=0)
full.to_csv('df_0_399.csv', index = False)

### Từ 400 đến 799


400 tới 499

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": '6bff30b75emshdd298c47c1c929ap1383f0jsn0c21e6195f9b',
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[400:500]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  404
F
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_500.csv',index = False)

Tu 500 den 599

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": '6f8d2ed624msh3e15216469ca746p12cc7ajsn072c037005c7',
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[500:600]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_600.csv',index = False)

Tu 600 den 699

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": 'bbf5f5b11emshaf127affd902405p12fe5ajsn829419d7abe6',
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[600:700]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  404
F
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  404
F
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_700.csv',index = False)

Tu 700 den 799

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "01cb255682mshe8a5b984adc38c4p168ac3jsnd76139a04888",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[700:800]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_800.csv',index = False)

In [ ]:
df_500 = pd.read_csv('df_500.csv', encoding = 'utf-8')
df_600 = pd.read_csv('df_600.csv', encoding = 'utf-8')
df_700 = pd.read_csv('df_700.csv', encoding = 'utf-8')
df_800 = pd.read_csv('df_800.csv', encoding = 'utf-8')
full = pd.concat([df_500,df_600,df_700,df_800],axis=0)
full.to_csv('df_400_799.csv', index = False)

### Tu 800 den 1199

800 den 900

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "9f5fe71373msh277bfa090fa9e3fp19c725jsn65170baf2bc9",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[800:900]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  404
F
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_900.csv',index = False)

900 toi 1000

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "80f5fe4f2bmshd2c8a43c16f29aep1e110fjsn1a258cb62e0f",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[900:1000]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1000.csv',index = False)

1000 toi 1100

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "b45e4843c0msh0f2c7367f96e63bp103ddajsnd1492df1a4ba",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1000:1100]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1100.csv',index = False)

1100 toi 1200

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "8d560cf78dmshafb63ed3984e7f0p11078ejsn14c9a26443a2",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1100:1200]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  404
F
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1200.csv',index = False)

In [ ]:
df_900 = pd.read_csv('df_900.csv', encoding = 'utf-8')
df_1000 = pd.read_csv('df_1000.csv', encoding = 'utf-8')
df_1100 = pd.read_csv('df_1100.csv', encoding = 'utf-8')
df_1200 = pd.read_csv('df_1200.csv', encoding = 'utf-8')
full = pd.concat([df_900,df_1000,df_1100,df_1200],axis=0)
full.to_csv('df_800_1199.csv', index = False)

### Tu 1200 den 1600

1200 den 1300

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "a701564729msh5f035b6bffd776ap1f4ff7jsncdf04de16691",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1200:1300]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  404
F
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1300.csv',index = False)

1300 toi 1400

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "f6162d6b19mshbb316394fafd2ddp10376fjsn74ba3f927013",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1300:1400]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  404
F
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1400.csv',index = False)

1400 toi 1500

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "b6af23bd98mshaaefcc5917a3b45p162425jsnd49637f22d70",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1400:1500]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  404
F
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1500.csv',index = False)

1500 toi 1600

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "34ebec4defmsh5fdced7a2c4389bp1a8703jsnfab38bc588d5",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1500:1600]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  404
F
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  404
F
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1600.csv',index = False)

In [ ]:
df_1300 = pd.read_csv('df_1300.csv', encoding = 'utf-8')
df_1400 = pd.read_csv('df_1400.csv', encoding = 'utf-8')
df_1500 = pd.read_csv('df_1500.csv', encoding = 'utf-8')
df_1600 = pd.read_csv('df_1600.csv', encoding = 'utf-8')
full = pd.concat([df_1300,df_1400,df_1500,df_1600],axis=0)
full.to_csv('df_1200_1599.csv', index = False)

### Thật ra có tới 1620 bài, nên nhóm chạy thêm 20 bài và không lưu 20 bài này mà chạy ngay rồi gộp với dữ liệu.

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "34ebec4defmsh5fdced7a2c4389bp1a8703jsnfab38bc588d5",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1600:]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})

In [ ]:
df_0_399 = pd.read_csv('df_0_399.csv', encoding = 'utf-8')
df_400_799 = pd.read_csv('df_400_799.csv', encoding = 'utf-8')
df_800_1199 = pd.read_csv('df_800_1199.csv', encoding = 'utf-8')
df_1200_1599 = pd.read_csv('df_1200_1599.csv', encoding = 'utf-8')
full = pd.concat([df_0_399,df_400_799,df_800_1199,df_1200_1599, new_df],axis=0)

Sau khi có được ba cột lyric, tag, tier, ta tiến hành gộp với dữ liệu sau khi sử dụng Genius API.

In [ ]:
spotify_genius = pd.read_csv('popular_gt_80.csv', encoding = 'utf-8')
full = pd.concat([spotify_genius,full], axis = 1)

In [ ]:
full.to_csv('1620_popular_songs.csv',index = False,encoding = "utf-8")

**Dữ liệu cuối cùng**: Sau khi tổng hợp cả 3 bước, file '1620_popular_songs.csv' sẽ là raw data được sử dụng cho những phần sau.

## Cào 10 bài nhạc để test:

In [ ]:
tracks_test = []
for i in range(0,10):
  tracks_test.append(sp.search(get_random_search(),offset= random.randint(0, 500),limit=10))

In [ ]:
track_df_test = pd.DataFrame()
for i in tracks_test:
  track_df_test = pd.concat([track_df_test,pd.json_normalize(i['tracks']['items'])],axis=0)

In [ ]:
track_ids_test = list(track_df_test.id.unique())

In [ ]:
track_ids_test

['5KfJvZ0PZzRdwFRaTUDAA7',
 '1qrpoAMXodY6895hGKoUpA',
 '4BP3uh0hFLFRb5cjsgLqDh',
 '1KGi9sZVMeszgZOWivFpxs',
 '5SWnsxjhdcEDc7LJjq9UHk',
 '0WyH4lAtIBPmkavC3uw5l4',
 '7EI5AsVGSHmMinLraUFdAt',
 '7snQQk1zcKl8gZ92AnueZW',
 '0otRX6Z89qKkHkQ9OqJpKt',
 '4NoOME4Dhf4xgxbHDT7VGe',
 '3Wrjm47oTz2sjIgck11l5e',
 '17cia5k4BpGVlyEfqrFH50',
 '69uxyAqqPIsUyTO8txoP2M',
 '1RMJOxR6GRPsBHL8qeC2ux',
 '2iVgM5C7m1G4CJGbms301G',
 '6K4t31amVTZDgR3sKmwUJJ',
 '5BKKy9fIJL5uM9fz1SnqyP',
 '7EZC6E7UjZe63f1jRmkWxt',
 '1F6IbA7di42uPc3cff8PXV',
 '4HwDCXsMBC7SUdp2WT4MZP',
 '3e1rs346dsDDwpqTRGlRZR',
 '3kf0WdFOalKWBkCCLJo4mA',
 '5LrN7yUQAzvthd4QujgPFr',
 '22VHOlVYBqytsrAqV8yXBK',
 '01cN1DKc5HH7F9MOBBjSip',
 '7xQAfvXzm3AkraOtGPWIZg',
 '63OQupATfueTdZMWTxW03A',
 '18lR4BzEs7e3qzc0KVkTpU',
 '6LSza8vBsCd7xLk13cYhIv',
 '3khLpwuyIjEOw8vfJ1oUIT',
 '6MF4tRr5lU8qok8IKaFOBE',
 '5zA8vzDGqPl2AzZkEYQGKh',
 '11IzgLRXV7Cgek3tEgGgjw',
 '32OlwWuMpZ6b0aN2RZOeMS',
 '5ixdmTpCmH5diBA1j154wN',
 '0ITaSRuI0L1RNL7ZNYTTi7',
 '1yvMUkIOTeUNtNWlWRgANS',
 

In [ ]:
features_df_test = []
for i in range(0,len(track_ids),50):
  features_df_test.append(sp.audio_features(track_ids_test[i:i+50]))

# Ví dụ output API trả về khi truy xuất thông tin các features nhạc từ id:

In [ ]:
sp.audio_features(track_ids_test[0])

[{'danceability': 0.872,
  'energy': 0.723,
  'key': 1,
  'loudness': -5.968,
  'mode': 1,
  'speechiness': 0.247,
  'acousticness': 0.0293,
  'instrumentalness': 0,
  'liveness': 0.327,
  'valence': 0.843,
  'tempo': 93.279,
  'type': 'audio_features',
  'id': '4VQNCzfZ3MdHEwwErNXpBo',
  'uri': 'spotify:track:4VQNCzfZ3MdHEwwErNXpBo',
  'track_href': 'https://api.spotify.com/v1/tracks/4VQNCzfZ3MdHEwwErNXpBo',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4VQNCzfZ3MdHEwwErNXpBo',
  'duration_ms': 307773,
  'time_signature': 4}]

In [ ]:
flattened_list = [item for sublist in features_df_test for item in sublist]
filtered_data_list = [data for data in flattened_list if data is not None and data]
features_df_test = pd.DataFrame(filtered_data_list)

In [ ]:
popularity = []
song_name = []
artist_name = []
for id in features_df_test['id']:
  track = sp.track(id)
  artist_name.append(track['artists'][0]['name'])
  song_name.append(track['name'])
  popularity.append(track['popularity'])

# Ví dụ output API trả về khi truy xuất thông tin 1 track từ id:

In [ ]:
#track_example = sp.track(track_ids_test[0])

In [ ]:
#track_example['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1ZwdS5xdxEREPySFridCfh'},
  'href': 'https://api.spotify.com/v1/artists/1ZwdS5xdxEREPySFridCfh',
  'id': '1ZwdS5xdxEREPySFridCfh',
  'name': '2Pac',
  'type': 'artist',
  'uri': 'spotify:artist:1ZwdS5xdxEREPySFridCfh'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/5J32qSlWzKCdcL31B8wglv'},
  'href': 'https://api.spotify.com/v1/artists/5J32qSlWzKCdcL31B8wglv',
  'id': '5J32qSlWzKCdcL31B8wglv',
  'name': 'Big Syke',
  'type': 'artist',
  'uri': 'spotify:artist:5J32qSlWzKCdcL31B8wglv'}]

In [ ]:
#track_example['name']

'All Eyez On Me (ft. Big Syke)'

In [ ]:
#track_example['popularity']

78

In [ ]:
features_df_test['song_name'] = song_name
features_df_test['popularity'] = popularity
features_df_test['artist_name'] = artist_name

In [ ]:
features_df_test.head(5)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,song_name,popularity,artist_name
0,0.872,0.723,1,-5.968,1,0.2470,0.0293,0.000000,0.3270,0.8430,...,audio_features,4VQNCzfZ3MdHEwwErNXpBo,spotify:track:4VQNCzfZ3MdHEwwErNXpBo,https://api.spotify.com/v1/tracks/4VQNCzfZ3MdH...,https://api.spotify.com/v1/audio-analysis/4VQN...,307773,4,All Eyez On Me (ft. Big Syke),78,2Pac
1,0.911,0.778,5,-2.248,1,0.2620,0.0887,0.000022,0.1790,0.3450,...,audio_features,7iQXYTyuG13aoeHxGG28Nh,spotify:track:7iQXYTyuG13aoeHxGG28Nh,https://api.spotify.com/v1/tracks/7iQXYTyuG13a...,https://api.spotify.com/v1/audio-analysis/7iQX...,162768,4,PERRO NEGRO,94,Bad Bunny
2,0.847,0.637,7,-8.751,0,0.0587,0.2010,0.201000,0.1210,0.0509,...,audio_features,5KTZgG84bKFGm53lhLtTqc,spotify:track:5KTZgG84bKFGm53lhLtTqc,https://api.spotify.com/v1/tracks/5KTZgG84bKFG...,https://api.spotify.com/v1/audio-analysis/5KTZ...,208135,4,Mwaki,74,Zerb
3,0.683,0.932,8,-10.402,1,0.0513,0.0389,0.605000,0.0615,0.5530,...,audio_features,0w7JPlp7eEQI2EKW3ayXrv,spotify:track:0w7JPlp7eEQI2EKW3ayXrv,https://api.spotify.com/v1/tracks/0w7JPlp7eEQI...,https://api.spotify.com/v1/audio-analysis/0w7J...,235786,4,Drugs From Amsterdam,75,Mau P
4,0.454,0.931,11,-4.083,1,0.0321,0.2380,0.000623,0.2890,0.4620,...,audio_features,3YfGTvsTAWGC2lgoDOikUz,spotify:track:3YfGTvsTAWGC2lgoDOikUz,https://api.spotify.com/v1/tracks/3YfGTvsTAWGC...,https://api.spotify.com/v1/audio-analysis/3YfG...,160530,4,One in a Million,78,Bebe Rexha


In [ ]:
features_df_test.to_csv("test.csv")